# Introduction to Using SEC Edgar Data with AlgoSeek Datasets

## 1) Introduction

This notebook demonstrates how to fetch sec edgar filing data and use it with AlgoSeek's Equity Data

In [ ]:
# import libraries
import os
import requests
import pandas as pd
import numpy as np
import seaborn as sns


In order to use the SEC Edgar API, you must use a valid email address. This is simply an anti-bot measure

In [ ]:
# headers = {'User-Agent': "your@email.com"}
headers = {'User-Agent': "julian@julianwiley.com"}


Fetching Financial data

In [ ]:
def get_financials(stock_cik):
    """

    :param stock_cik:
    :type stock_cik:
    :return:
    :rtype:
    """

    response = requests.get(f"https://data.sec.gov/api/xbrl/companyconcept/{stock_cik}/us-gaap/Assets.json".format(stock_cik=stock_cik), headers=headers)

    return response

In [ ]:
response = requests.get("https://data.sec.gov/api/xbrl/companyconcept/CIK0000320193/us-gaap/Assets.json", headers=headers)